In [516]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [517]:
path = '/Users/nick0o0o0/Library/Mobile Documents/com~apple~CloudDocs/gxyfile/third_year/ECO353/Assignment/project1/data/'

NBA_attend = pd.read_csv(path + 'NBA Attend Index.csv')
NBA_revenue = pd.read_csv(path+ 'Finance.csv')

In [518]:
NBA_revenue['Team'] = NBA_revenue['Team'].str.title()

In [519]:
team_city_dict = {'Vancouver Grizzlies': 'Vancouver', 'Phoenix': 'Phoenix', 'Dallas': 'Dallas', 'Memphis': 'Memphis', 'New York': 'New York', 'Orlando Magic': 'Orlando', 'Utah': 'Utah', 'Portland': 'Portland', 'Boston': 'Boston', 'Milwaukee': 'Milwaukee', 'Oklahoma City Thunder': 'Oklahoma City', 'Miami Heat': 'Miami', 'Indiana Pacers': 'Indiana', 'Boston Celtics': 'Boston', 'Portland Trail Blazers': 'Portland', 'Sacramento': 'Sacramento', 'Charlotte Bobcats': 'Charlotte', 'Denver': 'Denver', 'Toronto': 'Toronto', 'Washington Wizards': 'Washington', 'New York Knicks': 'New York', 'Houston Rockets': 'Houston', 'Orlando': 'Orlando', 'Seattle Supersonics': 'Seattle', 'Dallas Mavericks': 'Dallas', 'Cleveland Cavaliers': 'Cleveland', 'New Jersey Nets': 'New Jersey', 'Chicago': 'Chicago', 'Oklahoma City': 'Oklahoma City', 'San Antonio Spurs': 'San Antonio', 'Atlanta': 'Atlanta', 'Indiana': 'Indiana', 'Brooklyn Nets': 'Brooklyn', 'Philadelphia': 'Philadelphia', 'Charlotte': 'Charlotte', 'Milwaukee Bucks': 'Milwaukee', 'New Jersey': 'New Jersey', 'New Orleans Pelicans': 'New Orleans', 'Los Angeles Lakers': 'Los Angeles', 'Toronto Raptors': 'Toronto', 'Atlanta Hawks': 'Atlanta', 'Utah Jazz': 'Utah', 'Denver Nuggets': 'Denver', 'Sacramento Kings': 'Sacramento', 'Detroit': 'Detroit', 'Miami': 'Miami', 'Memphis Grizzlies': 'Memphis', 'San Antonio': 'San Antonio', 'Washington': 'Washington', 'Dallas Mavericks/': 'Dallas', 'League Average': 'N/A', 'New Orleans': 'New Orleans', 'Minnesota': 'Minnesota', 'Los Angeles Clippers': 'Los Angeles', 'Chicago Bulls': 'Chicago', 'New Orleans Hornets': 'New Orleans', 'Phoenix Suns': 'Phoenix', 'Detroit Pistons': 'Detroit', 'Charlotte Hornets': 'Charlotte', 'Minnesota Timberwolves': 'Minnesota', 'Golden State': 'San Francisco', 'Golden State Warriors': 'San Francisco', 'Houston': 'Houston', 'Cleveland': 'Cleveland', 'Philadelphia 76Ers': 'Philadelphia'}

NBA_revenue['City'] = NBA_revenue['Team'].map(team_city_dict)

In [520]:

City_pop_00 = pd.read_csv(path + 'Population00-10.csv')
City_pop_10 = pd.read_csv(path + 'Population10-20.csv')

In [521]:
NBA_revenue['Year2'] = NBA_revenue['Year'].str[-4:]

City_pop_00 = City_pop_00[City_pop_00['NAME'].str.endswith('city')]
City_pop_00 = City_pop_00.rename(columns={'POPESTIMATE2000': '2000',
                                          'POPESTIMATE2001': '2001',
                                          'POPESTIMATE2002': '2002',
                                          'POPESTIMATE2003': '2003',
                                          'POPESTIMATE2004': '2004',
                                          'POPESTIMATE2005': '2005',
                                          'POPESTIMATE2006': '2006',
                                          'POPESTIMATE2007': '2007',
                                          'POPESTIMATE2008': '2008',
                                          'POPESTIMATE2009': '2009',
                                          'POPESTIMATE2010': '2010'})

# Extract the city name from the 'NAME' column of the City_pop_00 dataframe
City_pop_00['City'] = City_pop_00['NAME'].str.split(', ').str[0].str.replace(' city$', '', regex=True)

# Select only the relevant columns from City_pop_00 dataframe
City_pop_00 = City_pop_00[['City', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']]

City_pop_10 = City_pop_10[City_pop_10['NAME'].str.endswith('city')]
City_pop_10 = City_pop_10.rename(columns={
    'POPESTIMATE2010': '2010',
    'POPESTIMATE2011': '2011',
    'POPESTIMATE2012': '2012',
    'POPESTIMATE2013': '2013',
    'POPESTIMATE2014': '2014',
    'POPESTIMATE2015': '2015',
    'POPESTIMATE2016': '2016',
    'POPESTIMATE2017': '2017',
    'POPESTIMATE2018': '2018',
    'POPESTIMATE2019': '2019',
    'POPESTIMATE2020': '2020'
})

# Extract the city name from the 'NAME' column of the City_pop_10 dataframe
City_pop_10['City'] = City_pop_10['NAME'].str.split(', ').str[0].str.replace(' city$', '', regex=True)

# Select only the relevant columns from City_pop_10 dataframe
City_pop_10 = City_pop_10[['City', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']]

City_pop_merged = pd.merge(City_pop_10, City_pop_00, on=['City'], how='inner')
City_pop_merged = pd.melt(City_pop_merged, id_vars=['City'], var_name='Year2', value_name='Population')
merged_data = pd.merge(NBA_revenue, City_pop_merged, on=['City', 'Year2'], how='inner')

In [522]:
merged_data = merged_data.groupby(['Team', 'Year2'], as_index=False)['Population'].max()

# Merge with NBA_revenue again to get the revenue data for each team and year
merged_data = pd.merge(NBA_revenue, merged_data, on=['Team', 'Year2'], how='left')

# Rename the merged columns to more descriptive names
merged_data = merged_data.rename(columns={'Year2': 'Year2', 'Population': 'Population_Year'})

# Fill any missing population data with 0
merged_data['Population_Year'] = merged_data['Population_Year'].fillna(0)

In [523]:
merged_data

,Year,Rank,Team,GateRevenue,OtherRevenue,TotalRevenues,PlayerExpenses,OtherExpenses,TotalExpenses,Operatingincome,City,Year2,Population_Year
0,2000-2001,NaN,Los Angeles Lakers,NaN,NaN,144.0,65.1,47.8,NaN,31.1,Los Angeles,2001,3733328.0
1,2000-2001,NaN,New York Knicks,NaN,NaN,157.0,81.6,46.9,NaN,28.5,New York,2001,8059813.0
2,2000-2001,NaN,Chicago Bulls,NaN,NaN,117.0,32.7,32.6,NaN,51.7,Chicago,2001,2874477.0
3,2000-2001,NaN,Portland Trail Blazers,NaN,NaN,101.0,89.5,30.8,NaN,-19.3,Portland,2001,535755.0
4,2000-2001,NaN,Philadelphia 76Ers,NaN,NaN,103.0,53.6,38.3,NaN,11.1,Philadelphia,2001,1505455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,2020-2021,26.0,Indiana Pacers,32.0,232.0,264.0,127.0,72.0,199.0,65.0,Indiana,2021,0.0
623,2020-2021,27.0,Charlotte Hornets,31.0,238.0,269.0,105.0,70.0,175.0,94.0,Charlotte,2021,0.0
624,2020-2021,28.0,Minnesota Timberwolves,44.0,227.0,271.0,134.0,50.0,184.0,87.0,Minnesota,2021,0.0
625,2020-2021,29.0,Memphis Grizzlies,44.0,228.0,272.0,93.0,72.0,165.0,107.0,Memphis,2021,0.0
